# Load

In this exercise, you'll load data into different formats: a csv file, a json file, and a SQLite database.

You'll work with the GDP, population, and projects data. Run the code cell below to read in and clean the World Bank population and gdp data. This code creates a dataframe called df_indicator with both the gdp and population data.

In [1]:
# run this code cell - there is nothing for you to do in this code cell

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# read in the projects data set and do basic wrangling 
gdp = pd.read_csv('../data/gdp_data.csv', skiprows=4)
gdp.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)
population = pd.read_csv('../data/population_data.csv', skiprows=4)
population.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)


# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=['Country Name', 'Country Code'], 
                    var_name='year', 
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby(['Country Name', 'Country Code'])['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name', 'Country Code'], 
                                  var_name='year', 
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')

# merge the population and gdp data together into one data frame
df_indicator = gdp_melt.merge(population_melt, on=('Country Name', 'Country Code', 'year'))

# filter out values that are not countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_indicator  = df_indicator[~df_indicator['Country Name'].isin(non_countries)]
df_indicator.reset_index(inplace=True, drop=True)

df_indicator.columns = ['countryname', 'countrycode', 'year', 'gdp', 'population']

# output the first few rows of the data frame
df_indicator.head()

,countryname,countrycode,year,gdp,population
0,Aruba,ABW,1960,1.916242e+09,54211.0
1,Afghanistan,AFG,1960,5.377778e+08,8996351.0
2,Angola,AGO,1960,1.857319e+10,5643182.0
3,Albania,ALB,1960,1.857319e+10,1608800.0
4,Andorra,AND,1960,1.857319e+10,13411.0


Run this code cell to read in the countries data set. This will create a data frame called df_projects containing the World Bank projects data. The data frame only has the 'id', 'countryname', 'countrycode', 'totalamt', and 'year' columns.

In [2]:
# run this code cell - there is nothing for you to do here 

!pip install pycountry
from pycountry import countries

# read in the projects data set with all columns type string
df_projects = pd.read_csv('../data/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)

df_projects['countryname'] = df_projects['countryname'].str.split(';').str.get(0)

# set up the libraries and variables
from collections import defaultdict
country_not_found = [] # stores countries not found in the pycountry library
project_country_abbrev_dict = defaultdict(str) # set up an empty dictionary of string values

# iterate through the country names in df_projects. 
# Create a dictionary mapping the country name to the alpha_3 ISO code
for country in df_projects['countryname'].drop_duplicates().sort_values():
    try: 
        # look up the country name in the pycountry library
        # store the country name as the dictionary key and the ISO-3 code as the value
        project_country_abbrev_dict[country] = countries.lookup(country).alpha_3
    except:
        # If the country name is not in the pycountry library, then print out the country name
        # And store the results in the country_not_found list
        country_not_found.append(country)
        
# run this code cell to load the dictionary

country_not_found_mapping = {'Co-operative Republic of Guyana': 'GUY',
             'Commonwealth of Australia':'AUS',
             'Democratic Republic of Sao Tome and Prin':'STP',
             'Democratic Republic of the Congo':'COD',
             'Democratic Socialist Republic of Sri Lan':'LKA',
             'East Asia and Pacific':'EAS',
             'Europe and Central Asia': 'ECS',
             'Islamic  Republic of Afghanistan':'AFG',
             'Latin America':'LCN',
              'Caribbean':'LCN',
             'Macedonia':'MKD',
             'Middle East and North Africa':'MEA',
             'Oriental Republic of Uruguay':'URY',
             'Republic of Congo':'COG',
             "Republic of Cote d'Ivoire":'CIV',
             'Republic of Korea':'KOR',
             'Republic of Niger':'NER',
             'Republic of Kosovo':'XKX',
             'Republic of Rwanda':'RWA',
              'Republic of The Gambia':'GMB',
              'Republic of Togo':'TGO',
              'Republic of the Union of Myanmar':'MMR',
              'Republica Bolivariana de Venezuela':'VEN',
              'Sint Maarten':'SXM',
              "Socialist People's Libyan Arab Jamahiriy":'LBY',
              'Socialist Republic of Vietnam':'VNM',
              'Somali Democratic Republic':'SOM',
              'South Asia':'SAS',
              'St. Kitts and Nevis':'KNA',
              'St. Lucia':'LCA',
              'St. Vincent and the Grenadines':'VCT',
              'State of Eritrea':'ERI',
              'The Independent State of Papua New Guine':'PNG',
              'West Bank and Gaza':'PSE',
              'World':'WLD'}

project_country_abbrev_dict.update(country_not_found_mapping)

df_projects['countrycode'] = df_projects['countryname'].apply(lambda x: project_country_abbrev_dict[x])

df_projects['boardapprovaldate'] = pd.to_datetime(df_projects['boardapprovaldate'])

df_projects['year'] = df_projects['boardapprovaldate'].dt.year.astype(str).str.slice(stop=4)

df_projects['totalamt'] = pd.to_numeric(df_projects['totalamt'].str.replace(',',""))

df_projects = df_projects[['id', 'countryname', 'countrycode', 'totalamt', 'year']]

df_projects.head()

    100% |████████████████████████████████| 10.0MB 3.0MB/s eta 0:00:01  9% |██▉                             | 901kB 30.4MB/s eta 0:00:01    85% |███████████████████████████▎    | 8.5MB 33.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for pycountry ... done
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry


,id,countryname,countrycode,totalamt,year
0,P162228,World,WLD,0,2018
1,P163962,Democratic Republic of the Congo,COD,200000000,2018
2,P167672,People's Republic of Bangladesh,BGD,58330000,2018
3,P158768,Islamic Republic of Afghanistan,AFG,20000000,2018
4,P161364,Federal Republic of Nigeria,NGA,100000000,2018


# Exercise Part 1

The first few cells in this workbook loaded and cleaned the World Bank Data. You now have two data frames:
* df_projects, which contain data from the projects data set
* df_indicator, which contain population and gdp data for various years

They both have country code variables. Note, however, that there could be countries represented in the projects data set that are not in the indicator data set and vice versus.

In this first exercise, merge the two data sets together using country code and year as common keys. When joining the data sets, keep all of the data in the df_projects dataframe even if there is no indicator data for that country code.

In [19]:
df_projects['countrycode'] = df_projects['countrycode'].astype('object')
df_indicator['countrycode'] = df_indicator['countrycode'].astype('object')

In [23]:
# TODO: merge the projects and indicator data frames together using countrycode and year as common keys
# Use a left join so that all projects are returned even if the country/year combination does not have 
# indicator data
df_indicator['countrycode'].dtype

df_merged = df_projects.merge(df_indicator, on=['countrycode', 'year'], how="left")

In [24]:
df_merged.head()

,id,countryname_x,countrycode,totalamt,year,countryname_y,gdp,population
0,P162228,World,WLD,0,2018,NaN,NaN,NaN
1,P163962,Democratic Republic of the Congo,COD,200000000,2018,NaN,NaN,NaN
2,P167672,People's Republic of Bangladesh,BGD,58330000,2018,NaN,NaN,NaN
3,P158768,Islamic Republic of Afghanistan,AFG,20000000,2018,NaN,NaN,NaN
4,P161364,Federal Republic of Nigeria,NGA,100000000,2018,NaN,NaN,NaN


If you look at the first few rows of the merged data set, you'll see NaN's for the indicator data. That is because the indicator data does not contain values for 2018. If you run the code cell below, you should get the following results:


||id | countryname_x	| countrycode	|totalamt	|year	|countryname_y	|gdp	| population|
|-|-|-|-|-|-|-|
|256|	P161982|	Hashemite Kingdom of Jordan|	JOR|	0|	2017|	Jordan	|4.006831e+10|	9702353.0|
|301|	P162407|	Hashemite Kingdom of Jordan|	JOR|	147700000|	2017|	Jordan|	4.006831e+10|	9702353.0|
|318|	P160103|	Hashemite Kingdom of Jordan|	JOR|	0|	2017|	Jordan	|4.006831e+10	|9702353.0|
|464|	P161905|	Hashemite Kingdom of Jordan|	JOR|	50000000|	2017	|Jordan	|4.006831e+10	|9702353.0|
|495|	P163387|	Hashemite Kingdom of Jordan|	JOR|	36100000|	2017	|Jordan	|4.006831e+10	|9702353.0|
|515|	P163241|	Hashemite Kingdom of Jordan|	JOR|	0|	2017|	Jordan|	4.006831e+10|	9702353.0|

In [26]:
# Run this code to check your work
df_merged[(df_merged['year'] == '2017') & (df_merged['countryname_y'] == 'Jordan')]

,id,countryname_x,countrycode,totalamt,year,countryname_y,gdp,population
256,P161982,Hashemite Kingdom of Jordan,JOR,0,2017,Jordan,4.006831e+10,9702353.0
301,P162407,Hashemite Kingdom of Jordan,JOR,147700000,2017,Jordan,4.006831e+10,9702353.0
318,P160103,Hashemite Kingdom of Jordan,JOR,0,2017,Jordan,4.006831e+10,9702353.0
464,P161905,Hashemite Kingdom of Jordan,JOR,50000000,2017,Jordan,4.006831e+10,9702353.0
495,P163387,Hashemite Kingdom of Jordan,JOR,36100000,2017,Jordan,4.006831e+10,9702353.0
515,P163241,Hashemite Kingdom of Jordan,JOR,0,2017,Jordan,4.006831e+10,9702353.0


# Exercise Part 2

Output the df_merged dataframe as a json file. You can use the pandas [to_json() method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_json.html).

In [27]:
# TODO: Output the df_merged data frame as a json file
# HINT: Pandas has a to_json() method
# HINT: use orient='records' to get one of the more common json formats
# HINT: be sure to specify the name of the json file you want to create as the first input into to_json

df_merged.to_json('df_merged.json', orient='records')

If you go to File->Open and then open the 17_load_exercise folder, you can see the json file you created.

The records should look like this (note that the order of the records might be different, but the format should look similar):
```
[{"id":"P162228","countryname_x":"World","countrycode":"WLD","totalamt":0,"year":"2018","countryname_y":null,"gdp":null,"population":null},{"id":"P163962","countryname_x":"Democratic Republic of the Congo","countrycode":"COD","totalamt":200000000,"year":"2018","countryname_y":null,"gdp":null,"population":null},{"id":"P167672","countryname_x":"People's Republic of Bangladesh","countrycode":"BGD","totalamt":58330000,"year":"2018","countryname_y":null,"gdp":null,"population":null},{"id":"P158768","countryname_x":"Islamic  Republic of Afghanistan","countrycode":"AFG","totalamt":20000000,"year":"2018","countryname_y":null,"gdp":null,"population":null},{"id":"P161364","countryname_x":"Federal Republic of Nigeria","countrycode":"NGA","totalamt":100000000,"year":"2018","countryname_y":null,"gdp":null,"population":null},{"id":"P161483","countryname_x":"Republic of Tunisia","countrycode":"TUN","totalamt":500000000,"year":"2018","countryname_y":null,"gdp":null,"population":null}
```



# Exercise Part 3

Output the df_merged dataframe as a csv file. You can use the pandas [to_csv() method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html).

In [28]:
# TODO: Output the df_merged data frame as a csv file
# HINT: The to_csv() method is similar to the to_json() method.
# HINT: If you do not want the data frame indices in your result, use index=False

df_merged.to_csv('df_merged.csv', index=False)

If you go to File->Open and then open the 17_load_exercise folder, you can see the csv file you created.

The records should look something like this:
```
id,countryname_x,countrycode,totalamt,year,countryname_y,gdp,population
P162228,World,WLD,0,2018,,,
P163962,Democratic Republic of the Congo,COD,200000000,2018,,,
P167672,People's Republic of Bangladesh,BGD,58330000,2018,,,
P158768,Islamic  Republic of Afghanistan,AFG,20000000,2018,,,
P161364,Federal Republic of Nigeria,NGA,100000000,2018,,,
P161483,Republic of Tunisia,TUN,500000000,2018,,,
P161885,Federal Republic of Nigeria,NGA,350000000,2018,,,
P162069,Federal Republic of Nigeria,NGA,225000000,2018,,,
P163540,Federal Republic of Nigeria,NGA,125000000,2018,,,
P163576,Lebanese Republic,LBN,329900000,2018,,,
P163721,Democratic Socialist Republic of Sri Lan,LKA,200000000,2018,,,
P164082,Federal Republic of Nigeria,NGA,400000000,2018,,,
P164783,Nepal,NPL,0,2018,,,
```

    

# Exercise Part 4

Output the df_merged dataframe as a sqlite database file. For this exercise, you can put all of the data as one table. In the next exercise, you'll create a database with multiple tables. 

In [33]:
import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

# TODO: output the df_merged dataframe to a SQL table called 'merged'.
# HINT: Use the to_sql() method
# HINT: Use the conn variable for the connection parameter
# HINT: You can use the if_exists parameter like if_exists='replace' to replace a table if it already exists

df_merged.to_sql('merged', conn, if_exists='replace')

Run the code cell below to make sure that the worldbank.db file was created and the merged table loaded. You should get an output that is formatted like this:

|id|countryname_x|countrycode|totalamt|year|countryname_y|gdp|population
|-|-|-|-|-|-|-|-|
|P162033|'Federative  Republic of Brazil'|BRA|125000000|2017|Brazil|2.055506e+12|209288278.0|

In [34]:
pd.read_sql('SELECT * FROM merged WHERE year = "2017" AND countrycode = "BRA"', con = conn).head()

,index,id,countryname_x,countrycode,totalamt,year,countryname_y,gdp,population
0,273,P162033,Federative Republic of Brazil,BRA,125000000,2017,Brazil,2.055506e+12,209288278.0
1,285,P158000,Federative Republic of Brazil,BRA,0,2017,Brazil,2.055506e+12,209288278.0
2,289,P163868,Federative Republic of Brazil,BRA,250000000,2017,Brazil,2.055506e+12,209288278.0
3,313,P152285,Federative Republic of Brazil,BRA,0,2017,Brazil,2.055506e+12,209288278.0
4,346,P147158,Federative Republic of Brazil,BRA,50000000,2017,Brazil,2.055506e+12,209288278.0


# Exercise Part 5

Output the data to a SQL database like in the previous exercise; however, this time, put the df_indicator data in one table and the df_projects data in another table. Call the df_indicator table 'indicator' and the df_projects table 'projects'.

In [35]:
import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

# TODO: output the df_merged dataframe to a SQL table called 'merged'.
# HINT: Use the to_sql() method
# HINT: Use the conn variable for the connection parameter
# HINT: You can use the if_exists parameter like if_exists='replace' to replace a table if it already exists

df_projects.to_sql('projects', conn, if_exists='replace')
df_indicator.to_sql('indicator', conn, if_exists='replace')

Run the code cell below to see if your code is working properly. The code cell below runs a SQL query against the worldbank.db database joining the indicator table with the projects table.

In [36]:
pd.read_sql('SELECT * FROM projects LEFT JOIN indicator ON \
projects.countrycode = indicator.countrycode AND \
projects.year = indicator.year WHERE \
projects.year = "2017" AND projects.countrycode = "BRA"', con = conn).head()

,index,id,countryname,countrycode,totalamt,year,index,countryname,countrycode,year,gdp,population
0,273,P162033,Federative Republic of Brazil,BRA,125000000,2017,12395,Brazil,BRA,2017,2.055506e+12,209288278.0
1,285,P158000,Federative Republic of Brazil,BRA,0,2017,12395,Brazil,BRA,2017,2.055506e+12,209288278.0
2,289,P163868,Federative Republic of Brazil,BRA,250000000,2017,12395,Brazil,BRA,2017,2.055506e+12,209288278.0
3,313,P152285,Federative Republic of Brazil,BRA,0,2017,12395,Brazil,BRA,2017,2.055506e+12,209288278.0
4,346,P147158,Federative Republic of Brazil,BRA,50000000,2017,12395,Brazil,BRA,2017,2.055506e+12,209288278.0


In [37]:
# commit any changes to the database and close the connection to the database
conn.commit()
conn.close()

# Exercise Part 6 (Challenge)

SQLite, as its name would suggest, is somewhat limited in its functionality. For example, the Alter Table command only allows you to change a table name or to add a new column to a table. You can't, for example, add a primary key to a table once the table is already created.

If you want more control over a sqlite3 database, it's better to use the sqlite3 library directly. Here is an example of how to use the sqlite3 library to create a table in the database, insert a value, and then run a SQL query on the database. Run the code cells below to see the example.

### Demo

In [38]:
# connect to the data base
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# drop the test table in case it already exists
cur.execute("DROP TABLE IF EXISTS test")

# create the test table including project_id as a primary key
cur.execute("CREATE TABLE test (project_id TEXT PRIMARY KEY, countryname TEXT, countrycode TEXT, totalamt REAL, year INTEGER);")

# insert a value into the test table
cur.execute("INSERT INTO test (project_id, countryname, countrycode, totalamt, year) VALUES ('a', 'Brazil', 'BRA', '100,000', 1970);")

# commit changes made to the database
conn.commit()

# select all from the test table
cur.execute("SELECT * FROM test")
cur.fetchall()

[('a', 'Brazil', 'BRA', '100,000', 1970)]

In [39]:
# commit any changes and close the data base
conn.close()

### Exercise

Now, it's your turn. Use the sqlite3 library to connect to the worldbank.db database. Then: 
* Create a table, called projects, for the projects data where the primary key is the id of each project. 
* Create another table, called gdp, that contains the gdp data. 
* And create another table, called population, that contains the population data.

Here is the schema for each table.
##### projects

* project_id text 
* countryname text 
* countrycode text
* totalamt real
* year integer

project_id is the primary key

##### gdp
* countryname text
* countrycode text
* year integer
* gdp real

(countrycode, year) is the primary key

##### population
* countryname text
* countrycode text
* year integer
* population integer

(countrycode, year) is the primary key

After setting up the tables, write code that inserts the data into each table. (Note that this database is not normalized. For example, countryname and countrycode are in all three tables. You could make another table with countrycode and countryname and then create a foreign key constraint in the projects, gdp, and population tables. If you'd like an extra challenge, create a country table with countryname and countrycode. Then create the other tables with foreign key constraints).

Follow the TODOs in the next few code cells to finish the exercise.

In [107]:
# connect to the data base
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# drop tables created previously to start fresh
cur.execute("DROP TABLE IF EXISTS test")
cur.execute("DROP TABLE IF EXISTS indicator")
cur.execute("DROP TABLE IF EXISTS projects")
cur.execute("DROP TABLE IF EXISTS gdp")
cur.execute("DROP TABLE IF EXISTS population")

# TODO create the projects table including project_id as a primary key
# HINT: Use cur.execute("SQL Query")

cur.execute("CREATE TABLE projects ( id varchar(10), countryname varchar(32), countrycode varchar(3), totalamt numeric, year int, PRIMARY KEY (id))")

# TODO: create the gdp table including (countrycode, year) as primary key
# HINT: To create a primary key on multiple columns, you can do this:
# CREATE TABLE tablename (columna datatype, columnb datatype, columnc dataype, PRIMARY KEY (columna, columnb));

# TODO: create the gdp table including (countrycode, year) as primary key

cur.execute("CREATE TABLE gdp (countryname varchar(32), countrycode varchar(3), year int, gdp numeric, PRIMARY KEY (countrycode, year) )")


cur.execute("CREATE TABLE population (countryname varchar(32), countrycode varchar(3), year int, pop numeric, PRIMARY KEY (countrycode, year) )")

# commit changes to the database. Do this whenever modifying a database
conn.commit()

In [108]:
# TODO:insert project values into the projects table
# HINT: Use a for loop with the pandas iterrows() method
# HINT: The iterrows() method returns two values: an index for each row and a tuple of values
# HINT: Some of the values for totalamt and year are NaN. Because you've defined
# year and totalamt as numbers, you cannot insert NaN as a value into those columns.
# When totaamt or year equal NaN, you'll need to change the value to something numeric
# like, for example, zero

for vals in df_projects.iterrows():

    sql_statement = "INSERT INTO projects (id, countryname, countrycode, totalamt, year) VALUES ('{}', '{}', '{}', {}, {})"\
          .format(vals[1][0], vals[1][1].replace("\'",""), vals[1][2], vals[1][3], vals[1][4].replace('nan', '0'))
    cur.execute(sql_statement)

# commit changes to the dataset after any changes are made
conn.commit()

In [110]:
# TODO: insert gdp values into the gdp table
df_indicator = df_indicator.fillna('0')

for vals in df_indicator.iterrows():

    sql_statement = "INSERT INTO gdp (countryname, countrycode, year, gdp) VALUES ('{}', '{}', {}, {})"\
          .format(vals[1][0].replace("\'",""), vals[1][1], vals[1][2], vals[1][3])
    cur.execute(sql_statement)

# commit changes to the dataset after any changes are made
conn.commit()

(0, countryname          Aruba
countrycode            ABW
year                  1960
gdp            1.91624e+09
population           54211
Name: 0, dtype: object)
(1, countryname    Afghanistan
countrycode            AFG
year                  1960
gdp            5.37778e+08
population     8.99635e+06
Name: 1, dtype: object)
(2, countryname         Angola
countrycode            AGO
year                  1960
gdp            1.85732e+10
population     5.64318e+06
Name: 2, dtype: object)
(3, countryname        Albania
countrycode            ALB
year                  1960
gdp            1.85732e+10
population      1.6088e+06
Name: 3, dtype: object)
(4, countryname        Andorra
countrycode            AND
year                  1960
gdp            1.85732e+10
population           13411
Name: 4, dtype: object)
(5, countryname    United Arab Emirates
countrycode                     ARE
year                           1960
gdp                     1.85732e+10
population                    92634
N

Name: 215, dtype: object)
(216, countryname       Zimbabwe
countrycode            ZWE
year                  1960
gdp            1.05299e+09
population     3.74737e+06
Name: 216, dtype: object)
(217, countryname         Aruba
countrycode           ABW
year                 1961
gdp            2.7021e+11
population          55438
Name: 217, dtype: object)
(218, countryname    Afghanistan
countrycode            AFG
year                  1961
gdp            5.48889e+08
population     9.16676e+06
Name: 218, dtype: object)
(219, countryname         Angola
countrycode            AGO
year                  1961
gdp            1.96483e+10
population     5.75302e+06
Name: 219, dtype: object)
(220, countryname        Albania
countrycode            ALB
year                  1961
gdp            1.96483e+10
population      1.6598e+06
Name: 220, dtype: object)
(221, countryname        Andorra
countrycode            AND
year                  1961
gdp            1.96483e+10
population           14375
Nam

Name: 430, dtype: object)
(431, countryname    South Africa
countrycode             ZAF
year                   1961
gdp             7.97284e+09
population      1.79207e+07
Name: 431, dtype: object)
(432, countryname         Zambia
countrycode            ZMB
year                  1961
gdp            6.96286e+08
population     3.14026e+06
Name: 432, dtype: object)
(433, countryname       Zimbabwe
countrycode            ZWE
year                  1961
gdp            1.09665e+09
population     3.87076e+06
Name: 433, dtype: object)
(434, countryname          Aruba
countrycode            ABW
year                  1962
gdp            5.46667e+08
population           56225
Name: 434, dtype: object)
(435, countryname    Afghanistan
countrycode            AFG
year                  1962
gdp            5.46667e+08
population     9.34587e+06
Name: 435, dtype: object)
(436, countryname         Angola
countrycode            AGO
year                  1962
gdp             1.9888e+10
population     5.866

Name: 649, dtype: object)
(650, countryname       Zimbabwe
countrycode            ZWE
year                  1962
gdp             1.1176e+09
population     3.99942e+06
Name: 650, dtype: object)
(651, countryname          Aruba
countrycode            ABW
year                  1963
gdp            7.51111e+08
population           56695
Name: 651, dtype: object)
(652, countryname    Afghanistan
countrycode            AFG
year                  1963
gdp            7.51111e+08
population     9.53395e+06
Name: 652, dtype: object)
(653, countryname         Angola
countrycode            AGO
year                  1963
gdp            2.15018e+10
population     5.98042e+06
Name: 653, dtype: object)
(654, countryname        Albania
countrycode            ALB
year                  1963
gdp            2.15018e+10
population     1.76262e+06
Name: 654, dtype: object)
(655, countryname        Andorra
countrycode            AND
year                  1963
gdp            2.15018e+10
population           1641

Name: 863, dtype: object)
(864, countryname    Yemen, Rep.
countrycode            YEM
year                  1963
gdp            9.42321e+09
population     5.44606e+06
Name: 864, dtype: object)
(865, countryname    South Africa
countrycode             ZAF
year                   1963
gdp             9.42321e+09
population      1.88993e+07
Name: 865, dtype: object)
(866, countryname         Zambia
countrycode            ZMB
year                  1963
gdp            7.18714e+08
population     3.34514e+06
Name: 866, dtype: object)
(867, countryname       Zimbabwe
countrycode            ZWE
year                  1963
gdp            1.15951e+09
population     4.13276e+06
Name: 867, dtype: object)
(868, countryname    Aruba
countrycode      ABW
year            1964
gdp            8e+08
population     57032
Name: 868, dtype: object)
(869, countryname    Afghanistan
countrycode            AFG
year                  1964
gdp                  8e+08
population     9.73136e+06
Name: 869, dtype: objec

...

Name: 11732, dtype: object)
(11733, countryname          Benin
countrycode            BEN
year                  2014
gdp            9.70743e+09
population     1.02867e+07
Name: 11733, dtype: object)
(11734, countryname    Burkina Faso
countrycode             BFA
year                   2014
gdp             1.23774e+10
population       1.7586e+07
Name: 11734, dtype: object)
(11735, countryname     Bangladesh
countrycode            BGD
year                  2014
gdp            1.72885e+11
population     1.59405e+08
Name: 11735, dtype: object)
(11736, countryname       Bulgaria
countrycode            BGR
year                  2014
gdp             5.6732e+10
population     7.22394e+06
Name: 11736, dtype: object)
(11737, countryname        Bahrain
countrycode            BHR
year                  2014
gdp            3.33877e+10
population      1.3364e+06
Name: 11737, dtype: object)
(11738, countryname    Bahamas, The
countrycode             BHS
year                   2014
gdp             1.09

Name: 11938, dtype: object)
(11939, countryname        Andorra
countrycode            AND
year                  2015
gdp            2.81149e+09
population           78014
Name: 11939, dtype: object)
(11940, countryname    United Arab Emirates
countrycode                     ARE
year                           2015
gdp                     3.58135e+11
population               9.1543e+06
Name: 11940, dtype: object)
(11941, countryname      Argentina
countrycode            ARG
year                  2015
gdp            5.94749e+11
population     4.34178e+07
Name: 11941, dtype: object)
(11942, countryname        Armenia
countrycode            ARM
year                  2015
gdp            1.05533e+10
population     2.91695e+06
Name: 11942, dtype: object)
(11943, countryname    American Samoa
countrycode               ASM
year                     2015
gdp                  6.59e+08
population              55537
Name: 11943, dtype: object)
(11944, countryname    Antigua and Barbuda
countrycode   

Name: 12142, dtype: object)
(12143, countryname    Virgin Islands (U.S.)
countrycode                      VIR
year                            2015
gdp                        3.765e+09
population                    107710
Name: 12143, dtype: object)
(12144, countryname        Vietnam
countrycode            VNM
year                  2015
gdp            1.93241e+11
population     9.35716e+07
Name: 12144, dtype: object)
(12145, countryname        Vanuatu
countrycode            VUT
year                  2015
gdp            7.37917e+08
population          264603
Name: 12145, dtype: object)
(12146, countryname          Samoa
countrycode            WSM
year                  2015
gdp            8.03986e+08
population          193759
Name: 12146, dtype: object)
(12147, countryname         Kosovo
countrycode            XKX
year                  2015
gdp            6.43995e+09
population      1.8018e+06
Name: 12147, dtype: object)
(12148, countryname    Yemen, Rep.
countrycode            YEM
year 

Name: 12358, dtype: object)
(12359, countryname    British Virgin Islands
countrycode                       VGB
year                             2016
gdp                         3.765e+09
population                      30661
Name: 12359, dtype: object)
(12360, countryname    Virgin Islands (U.S.)
countrycode                      VIR
year                            2016
gdp                        3.765e+09
population                    107510
Name: 12360, dtype: object)
(12361, countryname        Vietnam
countrycode            VNM
year                  2016
gdp            2.05276e+11
population     9.45691e+07
Name: 12361, dtype: object)
(12362, countryname        Vanuatu
countrycode            VUT
year                  2016
gdp            7.87943e+08
population          270402
Name: 12362, dtype: object)
(12363, countryname          Samoa
countrycode            WSM
year                  2016
gdp            7.86356e+08
population          195125
Name: 12363, dtype: object)
(12364, coun

Name: 12565, dtype: object)
(12566, countryname         Turkey
countrycode            TUR
year                  2017
gdp            8.51102e+11
population      8.0745e+07
Name: 12566, dtype: object)
(12567, countryname         Tuvalu
countrycode            TUV
year                  2017
gdp            3.97313e+07
population           11192
Name: 12567, dtype: object)
(12568, countryname       Tanzania
countrycode            TZA
year                  2017
gdp            5.20903e+10
population       5.731e+07
Name: 12568, dtype: object)
(12569, countryname         Uganda
countrycode            UGA
year                  2017
gdp            2.58911e+10
population      4.2863e+07
Name: 12569, dtype: object)
(12570, countryname        Ukraine
countrycode            UKR
year                  2017
gdp            1.12154e+11
population     4.48312e+07
Name: 12570, dtype: object)
(12571, countryname        Uruguay
countrycode            URY
year                  2017
gdp             5.6157e+10
p

In [103]:
# TODO: insert population values into the population table

df_indicator = df_indicator.fillna('0')

for vals in df_indicator.iterrows():

    sql_statement = "INSERT INTO population (countryname, countrycode, year, population) VALUES ('{}', '{}', {}, {})"\
          .format(vals[1][0].replace("\'",""), vals[1][1], vals[1][2], vals[1][4])
    cur.execute(sql_statement)

# commit changes to the dataset after any changes are made
conn.commit()

# commit changes to the dataset after any changes are made
conn.commit()

In [18]:
# run this command to see if your tables were loaded as expected
sqlquery = "SELECT * FROM projects JOIN gdp JOIN population ON projects.year = gdp.year AND projects.countrycode = gdp.countrycode AND projects.countrycode = population.countrycode AND projects.year=population.year;"
result = pd.read_sql(sqlquery, con=conn)
result.shape

(15558, 13)

If everything went well, the code above should output (15558, 13) as the shape of the resulting data frame.

In [19]:
# commit any changes and close the database
conn.commit()
conn.close()

# Conclusion

Once you have extracted data and transformed it, the final step is to load the data into storage. In this exercise, you stored results into a csv file, a json file, and a SQLite database.